In [83]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import glob
import os
import datetime as dt
from datetime import datetime
import bz2
import pickle
import _pickle as cPickle

%matplotlib inline

In [29]:
tickers = pd.read_csv('ndx.csv', header=None)

In [39]:
tickers = tickers[0].to_list()

AttributeError: 'str' object has no attribute 'to_list'

In [41]:
len(tickers)

103

In [82]:
#tickers = ['FB', 'AA', 'AAPL']
stocks = download_tickers(['AAPL', 'MSFT'])

[*********************100%***********************]  2 of 2 completed
saved 0 lines
saved 1325 lines


In [80]:
stocks

Stock ticker        AAPL                                                  \
Data type           Open        High         Low       Close   Adj Close   
1972-06-01           NaN         NaN         NaN         NaN         NaN   
1972-06-02           NaN         NaN         NaN         NaN         NaN   
1972-06-05           NaN         NaN         NaN         NaN         NaN   
1972-06-06           NaN         NaN         NaN         NaN         NaN   
1972-06-07           NaN         NaN         NaN         NaN         NaN   
...                  ...         ...         ...         ...         ...   
2020-08-31    127.580002  131.000000  126.000000  129.039993  129.039993   
2020-09-01    132.759995  134.800003  130.529999  134.179993  134.179993   
2020-09-02    137.589996  137.979996  127.000000  131.399994  131.399994   
2020-09-03    126.910004  128.839996  120.500000  120.879997  120.879997   
2020-09-04    120.070000  123.699997  110.889999  120.959999  120.959999   

Stock ticker                     MSFT                                      \
Data type          Volume        Open        High         Low       Close   
1972-06-01            NaN         NaN         NaN         NaN         NaN   
1972-06-02            NaN         NaN         NaN         NaN         NaN   
1972-06-05            NaN         NaN         NaN         NaN         NaN   
1972-06-06            NaN         NaN         NaN         NaN         NaN   
1972-06-07            NaN         NaN         NaN         NaN         NaN   
...                   ...         ...         ...         ...         ...   
2020-08-31    225702700.0  227.000000  228.699997  224.309998  225.529999   
2020-09-01    152470100.0  225.509995  227.449997  224.429993  227.270004   
2020-09-02    200119000.0  227.970001  232.860001  227.350006  231.649994   
2020-09-03    254723200.0  229.270004  229.309998  214.960007  217.300003   
2020-09-04    332607163.0  215.100006  218.359894  205.190002  214.250000   

Stock ticker                          
Data type      Adj Close      Volume  
1972-06-01           NaN         NaN  
1972-06-02           NaN         NaN  
1972-06-05           NaN         NaN  
1972-06-06           NaN         NaN  
1972-06-07           NaN         NaN  
...                  ...         ...  
2020-08-31    225.529999  28774200.0  
2020-09-01    227.270004  25791200.0  
2020-09-02    231.649994  34080800.0  
2020-09-03    217.300003  58148900.0  
2020-09-04    214.250000  56783337.0  

[12178 rows x 12 columns]

In [66]:
stocks.loc[:, stocks.columns.get_level_values(1).isin(['Adj Close'])].isna().sum()

Stock ticker  Data type
AAPL          Adj Close     2160
AMZN          Adj Close     6311
MSFT          Adj Close     3485
FB            Adj Close    10089
GOOGL         Adj Close     8137
                           ...  
WDC           Adj Close     1624
FOXA          Adj Close    11801
LBTYK         Adj Close     8403
FOX           Adj Close    11802
LBTYA         Adj Close     8084
Length: 103, dtype: int64

In [68]:
def download_tickers(tickers):
    # Fetch downloaded ticker from raw_data
    downloaded_tickers = [x[0:-5] if '.json' in x else '' for x in os.listdir('raw_data/')]
    downloaded_tickers.remove('')
    # Check if tickers already are downloaded
    tickers_to_download = []
    for ticker in tickers:
        if ticker in downloaded_tickers:
            # If already downloaded, check if they need updates
            if update_ticker(ticker):
                tickers_to_download.append(ticker)
        else:
            tickers_to_download.append(ticker)

    download_dump(tickers_to_download)
    
    # Read tickers from json and return
    return load_stocks(tickers)

In [69]:
def load_stocks(tickers):
    cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    index = pd.MultiIndex.from_product([tickers, cols], names=['Stock ticker', 'Data type'])
    data = pd.concat([pd.read_json('raw_data/' + x + '.json', ) for x in tickers], axis=1, sort=True)
    data.columns = index
    try:
        data.index = pd.to_datetime(data.index, unit='ms')
    except ValueError:
        data.index = pd.to_datetime(data.index)
    return data

In [70]:
def update_ticker(ticker):
    last_bday = last_weekday()
    newest_date = pd.read_json('raw_data/' + ticker + '.json').index[-1]
    if newest_date == float:
        newest_date = dt.datetime.fromtimestamp(newest_date/1000).date()
    if newest_date != last_bday:
        return True 
    else:
        return False

In [76]:
def download_dump(tickers):
    if tickers != []:
        data = yf.download(tickers, period='100y', group_by='tickers')
        if len(tickers) > 1:
            for ticker in tickers:
                    clean_data = clean_df(data[ticker])
                    clean_data.to_json('raw_data/' + ticker + '.json', date_format='epoch', date_unit='ms')
        else:
            clean_data = clean_df(data)
            clean_data.to_json('raw_data/' + tickers[0] + '.json', date_format='epoch', date_unit='ms')

In [77]:
def last_weekday():
    todays_day = dt.date.today().day
    if todays_day in [5, 6]:
        return dt.date.today() - dt.timedelta(days=todays_day-4)
    else:
        return dt.date.today()


In [81]:
def clean_df(df):
    before = len(df)
    df = df.dropna(axis = 0, how='all')
    print('saved', before-len(df), 'lines')
    return df

In [84]:
def compressed_pickle(ticker, data):
    with bz2.BZ2File('raw_data' + ticker + '.pbz2', 'w') as f: 
        cPickle.dump(data, f)

In [85]:
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data